## Creating a mesh

We can also use the Splatter wrapper class to take an existing nerfstudio model and create a mesh!
1. **mesh:** creates a mesh via TSDF fusion

2. **query_mesh:** uses the trained model to query the mesh and returns a similarity map

3. **plot_mesh:** enables plotting of mesh features



In [1]:
import os, sys
from pathlib import Path
from collab_splats.wrapper import Splatter, SplatterConfig
import pyvista as pv

pv.start_xvfb()


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


/opt/conda/envs/nerfstudio/lib/python3.10/site-packages/pyvista/plotting/utilities/xvfb.py:48: PyVistaDeprecationWarning: This function is deprecated and will be removed in future version of PyVista. Use vtk-osmesa instead.
  warnings.warn(


Set paths to the file for running splats

In [2]:
base_dir = Path('/workspace/fieldwork-data/')
session_dir = base_dir / "birds/2024-02-06/SplatsSD"

# Make the configuration 
splatter_config = SplatterConfig(
    file_path=session_dir / "C0043.MP4",
    method='rade-features',
    frame_proportion=0.25, # Use 25% of the frames within the video (or default to minimum 300 frames)
)

# Initialize the Splatter class
splatter = Splatter(splatter_config)

# Call these to populate the splatter with paths (probably a better way to do this --> maybe save out config)
splatter.preprocess()
splatter.extract_features()

transforms.json already exists at /workspace/fieldwork-data/birds/2024-02-06/environment/C0043/preproc/transforms.json
To rerun preprocessing, set overwrite=True
Output already exists for rade-features
To rerun feature extraction, set overwrite=True


### Create a mesh

We can create a mesh by calling the ```mesh()``` method. Under the hood, this runs TSDF fusion creating an integrated volume. 

In [ ]:
mesher_kwargs = {
    'depth_name': "median_depth",
    'depth_trunc': 1.0, # Should be between 1.0 and 3.0
    'voxel_size': 0.005, 
    'normals_name': "normals",
    'features_name': "distill_features", 
    'sdf_trunc': 0.03,
    'clean_repair': True,
    'align_floor': True,
}

splatter.mesh(
    mesher_type="Open3DTSDFFusion",
    mesher_kwargs=mesher_kwargs,
    overwrite=True
)


Available runs:
[0] 2025-07-25_040743


[Taichi] version 1.7.3, llvm 15.0.4, commit 5ec301be, linux, python 3.10.18


[I 07/25/25 14:58:39.546 172106] [shell.py:_shell_pop_print@23] Graphical python shell detected, using wrapped sys.stdout


Initializing mesher Open3DTSDFFusion


[14:58:45] Auto image downscale factor of 2                                                 ]8;id=696833;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/data/dataparsers/nerfstudio_dataparser.py\nerfstudio_dataparser.py]8;;\:]8;id=650601;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/data/dataparsers/nerfstudio_dataparser.py#484\484]8;;\

Train dataset has over 500 images, overriding cache_images to cpu

[14:59:13] use color only optimization with sigmoid activation                                         ]8;id=412890;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/models/splatfacto.py\splatfacto.py]8;;\:]8;id=242121;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/models/splatfacto.py#266\266]8;;\

### Plot the mesh!

We can use the splatter function ```plot_mesh``` to visualize given attributes of the mesh. The inherent attributes are RGB and Normals

In [8]:
splatter.plot_mesh(attribute="RGB")

Number of points: 472386
Number of cells: 921526
Bounds: BoundsTuple(x_min=-1.0503028631210327, x_max=0.6878593564033508, y_min=-0.27155864238739014, y_max=2.515613317489624, z_min=-0.30862194299697876, z_max=1.018968105316162)


Widget(value='<iframe src="http://localhost:42247/index.html?ui=P_0x7449f98572e0_2&reconnect=auto" class="pyvi…

### Using semantic queries 

The mesh contains semantic features which we can query via positive and negative prompts. The goal of this is to find points that are more similar to the positive prompts compared to the negative prompts

In [5]:
similarity = splatter.query_mesh(
    positive_queries=["tree"],
    negative_queries=["ground", "leaves"],
    output_fn="query-tree.ply"
)

Loading model from /workspace/fieldwork-data/rats/2024-07-11/environment/C0119/rade-features/2025-07-11_171420/config.yml
[Taichi] version 1.7.3, llvm 15.0.4, commit 5ec301be, linux, python 3.10.18


[I 07/24/25 19:34:22.549 83051] [shell.py:_shell_pop_print@23] Graphical python shell detected, using wrapped sys.stdout


[19:34:26] Auto image downscale factor of 2                                                 ]8;id=736051;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/data/dataparsers/nerfstudio_dataparser.py\nerfstudio_dataparser.py]8;;\:]8;id=144982;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/data/dataparsers/nerfstudio_dataparser.py#484\484]8;;\

[19:35:00] use color only optimization with sigmoid activation                                         ]8;id=205326;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/models/splatfacto.py\splatfacto.py]8;;\:]8;id=890929;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/models/splatfacto.py#266\266]8;;\

Loading latest checkpoint from load_dir

✅ Done loading checkpoint from 
/workspace/fieldwork-data/rats/2024-07-11/environment/C0119/rade-features/2025-07-11_171420/nerfstudio_models/step-00002
9999.ckpt

Plot similarity maps

In [7]:
splatter.plot_mesh(attribute=similarity, rgb=False)

Number of points: 590469
Number of cells: 1165391
Bounds: BoundsTuple(x_min=-1.1681687831878662, x_max=0.9518513679504395, y_min=-0.3565163016319275, y_max=1.4750611782073975, z_min=-0.9717031121253967, z_max=-0.1970834583044052)


Widget(value='<iframe src="http://localhost:43259/index.html?ui=P_0x7b4aa0c8a230_1&reconnect=auto" class="pyvi…